# Add number of species to precalculated tables

In this notebook, we add the to each geometry with precalculated data, the number of species in each taxonomic group (amph_nspecies, bird_nspecies, mamm_nspecies, rept_nspecies) and the total number of species (nspecies).

## Table of contents
1. [Import packages](#packages)
2. [nspecies to gadm00](#gadm0)
3. [nspecies to gadm1](#gadm1)
4. [nspecies to future places](#places)
5. [nspecies to wdpa](#wdpa)

---
<a id='packages'></a>
## Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

---
<a id='gadm0'></a>
## Add nspecies to gadm0

In [6]:
# Import gadm layer used in production
gadm0 = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/gadm0_precalculated_20220224_nspecies_translated_backup.geojson')

In [7]:
gadm0.head(1)

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,percent_rangeland,percent_urban,country_size,ObjectId,alternative_names,nspecies,NAME_0_es,NAME_0_fr,NAME_0_pt,geometry
0,ABW,Aruba,1,181.9384,"[ { ""SliceNumber"": 1819, ""per_global"": 0.59999...","[ { ""SliceNumber"": 142, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 329, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 353, ""per_global"": 0.01, ""p...",20.6093,115656.1,...,None,6.39,5,1,None,163,Aruba,Aruba,Aruba,"POLYGON ((-69.97820 12.46986, -70.02847 12.503..."


In [8]:
gadm0.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'alternative_names', 'nspecies', 'NAME_0_es', 'NAME_0_fr', 'NAME_0_pt',
       'Shape__Area', 'Shape__Length', 'SHAPE'],
      dtype='object')

This layer already included nspecies field but we are going to remove them and include it again together with the number of species per taxonomic group

In [8]:
gadm0 = gadm0.drop(columns={'nspecies'})
gadm0.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'alternative_names', 'NAME_0_es', 'NAME_0_fr', 'NAME_0_pt', 'geometry'],
      dtype='object')

In [9]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_amphibians.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_birds.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_mammals.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_reptiles.csv')

In [10]:
amphibians.head(15)

,OBJECTID,MOL_ID,SliceNumber,FREQUENCY,SUM
0,1,1,1819,1,151.0
1,2,1,4583,1,95.0
2,3,1,5543,1,152.0
3,4,2,32,4,317.0
4,5,2,950,43,44105.0
5,6,2,951,36,10762.0
6,7,2,953,11,1186.0
7,8,2,1191,1,1.0
8,9,2,1707,56,182919.0
9,10,2,1935,46,50094.0


In [11]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [12]:
frame = { 'amph_nspecies': amph, 'bird_nspecies': bir, 'mamm_nspecies': mamm, 'rept_nspecies': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['amph_nspecies', 'bird_nspecies', 'mamm_nspecies', 'rept_nspecies']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['amph_nspecies'] + df['bird_nspecies'] + df['mamm_nspecies'] + df['rept_nspecies']
df

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,3,82,48,30,163
1,2,8,316,140,134,598
2,3,128,891,322,335,1676
3,4,2,96,6,10,114
4,5,5,88,26,2,121
...,...,...,...,...,...,...
250,251,0,36,0,3,39
251,252,6,218,71,129,424
252,253,117,706,271,399,1493
253,254,93,695,275,218,1281


In [13]:
gadm0_updated = gadm0.merge(df, how='left', on = 'MOL_ID')
gadm0_updated

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,alternative_names,NAME_0_es,NAME_0_fr,NAME_0_pt,geometry,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,ABW,Aruba,1,1.819384e+02,"[ { ""SliceNumber"": 1819, ""per_global"": 0.59999...","[ { ""SliceNumber"": 142, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 329, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 353, ""per_global"": 0.01, ""p...",20.609300,115656.10,...,None,Aruba,Aruba,Aruba,"POLYGON ((-69.97820 12.46986, -70.02847 12.503...",3,82,48,30,163
1,AFG,Afghanistan,2,6.438575e+05,"[ { ""SliceNumber"": 32, ""per_global"": 100.0, ""p...","[ { ""SliceNumber"": 26, ""per_global"": 1.4399999...","[ { ""SliceNumber"": 167, ""per_global"": 11.15, ""...","[ { ""SliceNumber"": 1, ""per_global"": 28.7100000...",3.785337,30389700.00,...,None,Afganistán,Afghanistan,Afeganistão,"POLYGON ((68.52644 31.75435, 68.53852 31.75457...",8,316,140,134,598
2,AGO,Angola,3,1.247422e+06,"[ { ""SliceNumber"": 2, ""per_global"": 0.58999999...","[ { ""SliceNumber"": 26, ""per_global"": 4.6699999...","[ { ""SliceNumber"": 28, ""per_global"": 2.2200000...","[ { ""SliceNumber"": 40, ""per_global"": 26.68, ""p...",6.679349,36074940.00,...,None,Angola,Angola,Angola,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -...",128,891,322,335,1676
3,AIA,Anguilla,4,8.330331e+01,"[ { ""SliceNumber"": 1819, ""per_global"": 0.23000...","[ { ""SliceNumber"": 142, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 329, ""per_global"": 0.0, ""pe...","[ { ""SliceNumber"": 316, ""per_global"": 25.32, ""...",2.880000,13601.96,...,None,Anguila,Anguilla,Anguila,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ...",2,96,6,10,114
4,ALA,Åland,5,1.506261e+03,"[ { ""SliceNumber"": 963, ""per_global"": 0.01, ""p...","[ { ""SliceNumber"": 116, ""per_global"": 0.01, ""p...","[ { ""SliceNumber"": 129, ""per_global"": 0.01, ""p...","[ { ""SliceNumber"": 2500, ""per_global"": 0.01, ""...",1.449518,29158.96,...,None,Åland,Åland,Ilhas Åland,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59...",5,88,26,2,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,XSP,Spratly Islands,251,1.874750e+00,None,"[ { ""SliceNumber"": 141, ""per_global"": 0.0, ""pe...",None,"[ { ""SliceNumber"": 3883, ""per_global"": 0.0, ""p...",0.000000,0.00,...,None,Islas Spratly,Îles Spratleys,Ilhas Spratly,"MULTIPOLYGON (((114.35472 11.45139, 114.35167 ...",0,36,0,3,39
251,YEM,Yemen,252,4.522034e+05,"[ { ""SliceNumber"": 974, ""per_global"": 74.01000...","[ { ""SliceNumber"": 26, ""per_global"": 0.2600000...","[ { ""SliceNumber"": 33, ""per_global"": 14.119999...","[ { ""SliceNumber"": 9, ""per_global"": 2.12000000...",0.000000,30204360.00,...,None,Yemen,Yémen,Iémen,"MULTIPOLYGON (((53.26931 12.12875, 53.31011 12...",6,218,71,129,424
252,ZAF,South Africa,253,1.219710e+06,"[ { ""SliceNumber"": 33, ""per_global"": 43.920000...","[ { ""SliceNumber"": 26, ""per_global"": 3.1499999...","[ { ""SliceNumber"": 28, ""per_global"": 3.9100000...","[ { ""SliceNumber"": 40, ""per_global"": 0.9499999...",8.157492,59911770.00,...,None,Sudáfrica,Afrique du Sud,África do Sul,"MULTIPOLYGON (((19.42097 -34.67903, 19.42208 -...",117,706,271,399,1493
253,ZMB,Zambia,254,7.504919e+05,"[ { ""SliceNumber"": 34, ""per_global"": 36.770000...","[ { ""SliceNumber"": 26, ""per_global"": 3.52, ""pe...","[ { ""SliceNumber"": 28, ""per_global"": 10.720000...","[ { ""SliceNumber"": 40, ""per_global"": 11.6, ""pe...",39.120460,18692100.00,...,None,Zambia,Zambie,Zâmbia,"POLYGON ((25.88753 -17.98407, 25.88284 -17.984...",93,695,275,218,1281


In [14]:
gadm0_updated[gadm0_updated.nspecies==0]

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,alternative_names,NAME_0_es,NAME_0_fr,NAME_0_pt,geometry,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies


In [15]:
gadm0_updated.to_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/updates/gadm0_precalculated_20220224_nspecies_all_translated.geojson', driver='GeoJSON')

Upload this layer manually in AGOL, either as a new feature layer or overwriting an existing service

---
<a id='gadm1'></a>
## Add nspecies to gadm1
The nspecies field for gadm1 were already included in the notebook Precalculated_gadm1, in which the precalculations had to be rerun due to errors with the birds global range

---
<a id='places'></a>
## Add nspecies to future places

In [11]:
# Import layer used in production
places = gpd.read_file('/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/top_places_precalculated_nspecies.geojson')

In [12]:
places.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,reptiles,majority_land_cover_climate_regime,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected,nspecies,geometry
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,1,...,"[ { ""SliceNumber"": 9.0, ""per_global"": 0.289999...",184.0,26.31,57.02,15.5,0.0,2.092001e+06,0.0,296,"POLYGON ((7827944.398 4014125.022, 7814867.987..."


In [13]:
places.columns

Index(['OBJECTID', 'GID_0', 'NAME_0', 'cluster', 'PRmean', 'AREA_KM2',
       'MOL_ID', 'Shape_Leng', 'Shape_Area', 'group', 'FID', 'amphibians',
       'birds', 'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'percent_irrigated', 'percent_rainfed', 'percent_rangeland',
       'percent_urban', 'population_sum', 'percentage_protected', 'nspecies',
       'geometry'],
      dtype='object')

In [18]:
places.nspecies

0       296
1       294
2       227
3       112
4       100
       ... 
1916    381
1917    165
1918    235
1919    204
1920    243
Name: nspecies, Length: 1921, dtype: int64

In [19]:
places = places.drop(columns='nspecies')
places.columns

Index(['OBJECTID', 'GID_0', 'NAME_0', 'cluster', 'PRmean', 'AREA_KM2',
       'MOL_ID', 'Shape_Leng', 'Shape_Area', 'group', 'FID', 'amphibians',
       'birds', 'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'percent_irrigated', 'percent_rainfed', 'percent_rangeland',
       'percent_urban', 'population_sum', 'percentage_protected', 'geometry'],
      dtype='object')

In [14]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_amph_nozeroes.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_birds_nozeroes.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_mammals_nozeroes.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_reptiles_nozeroes.csv')

In [15]:
amphibians.head()

,OID_,MOL_ID,X,Y,amphibians,SliceNumber,Dimensions
0,819,124,1.044409e+06,5.381166e+06,34.0,819.0,SliceNumber
1,956,124,1.044409e+06,5.381166e+06,1257.0,956.0,SliceNumber
2,963,124,1.044409e+06,5.381166e+06,2758.0,963.0,SliceNumber
3,2340,124,1.044409e+06,5.381166e+06,1515.0,2340.0,SliceNumber
4,2764,124,1.044409e+06,5.381166e+06,2883.0,2764.0,SliceNumber


In [16]:
reptiles.columns

Index(['OID_', 'MOL_ID', 'X', 'Y', 'reptiles', 'SliceNumber', 'Dimensions'], dtype='object')

In [17]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [20]:
frame = { 'amph_nspecies': amph, 'bird_nspecies': bir, 'mamm_nspecies': mamm, 'rept_nspecies': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['amph_nspecies', 'bird_nspecies', 'mamm_nspecies', 'rept_nspecies']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['amph_nspecies'] + df['bird_nspecies'] + df['mamm_nspecies'] + df['rept_nspecies']
df

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,5,174,65,52,296
1,2,4,192,66,32,294
2,3,2,144,48,33,227
3,4,0,67,27,18,112
4,5,0,62,23,15,100
...,...,...,...,...,...,...
1911,1921,34,364,100,82,580
1912,1922,26,354,89,84,553
1913,1923,34,414,112,89,649
1914,1924,26,326,93,76,521


In [21]:
df[df.nspecies==0]

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies


In [23]:
places_updated = places.merge(df, how='left', on = 'MOL_ID')
places_updated

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,percent_rangeland,percent_urban,population_sum,percentage_protected,geometry,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,AFG,Afghanistan,1,60.222222,7687.504920,1,442334.303716,1.124109e+10,1,...,15.50,0.00,2.092001e+06,0.00,"POLYGON ((7827944.398 4014125.022, 7814867.987...",5.0,174.0,65.0,52.0,296.0
1,2,AFG,Afghanistan,2,60.222222,4706.088317,2,354254.657577,7.043484e+09,1,...,5.24,0.00,6.288113e+05,0.13,"POLYGON ((7884416.986 4117164.050, 7880408.999...",4.0,192.0,66.0,32.0,294.0
2,3,AFG,Afghanistan,3,52.312500,11768.366965,3,647899.188567,1.746601e+10,1,...,111.57,2.66,6.122301e+06,0.03,"POLYGON ((7661980.648 3995094.171, 7659753.669...",2.0,144.0,48.0,33.0,227.0
3,4,AFG,Afghanistan,4,45.500000,205.123344,4,63354.536671,2.958675e+08,1,...,1.77,0.00,3.773772e+03,0.71,"POLYGON ((7551032.457 3954283.587, 7548339.754...",0.0,67.0,27.0,18.0,112.0
4,5,AFG,Afghanistan,5,45.500000,205.123344,5,63647.846097,2.979331e+08,1,...,2.64,0.00,1.882826e+03,0.00,"POLYGON ((7523202.584 3994616.263, 7520509.882...",0.0,62.0,23.0,15.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,1796,URY,Uruguay,1800,62.000000,205.123344,1800,61984.283090,2.861840e+08,9,...,14.99,0.00,9.704867e+05,0.00,"POLYGON ((-6169094.783 -3775236.665, -6171787....",21.0,252.0,50.0,58.0,381.0
1917,1797,UZB,Uzbekistan,1801,82.000000,205.123344,1801,71300.429216,3.510649e+08,9,...,9.31,2.68,2.342898e+06,0.00,"POLYGON ((6938775.258 4870286.877, 6936082.555...",0.0,104.0,35.0,26.0,165.0
1918,1798,UZB,Uzbekistan,1802,47.222222,6250.745122,1802,476428.050484,1.072342e+10,9,...,12.61,1.31,3.457537e+06,0.69,"POLYGON ((7969630.322 4820536.531, 7965575.423...",1.0,161.0,44.0,29.0,235.0
1919,1799,UZB,Uzbekistan,1803,39.500000,862.495051,1803,132210.652673,1.385523e+09,9,...,0.05,0.00,1.298762e+05,0.00,"POLYGON ((7597555.247 4577904.570, 7605992.591...",0.0,121.0,45.0,38.0,204.0


In [24]:
places_updated.to_file('/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/top_places_precalculated_nspecies_all.geojson', driver='GeoJSON')

---
<a id='wdpa'></a>
## Add nspecies to WDPA

In [2]:
# Import layer used in production
wdpa = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs/wdpa_precal_with_contextual_updated.csv')

In [3]:
wdpa.head(1)

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,0.0,100.0,0.0,2.110001,310492,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency


In [4]:
wdpa.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum', 'WDPA_PID', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP'],
      dtype='object')

In [3]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_amphibians_final_20211003.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_birds_final_20211003.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_mammals_final_20211003.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_reptiles_final_20211003.csv')

In [7]:
amphibians.head()

,OBJECTID,MOL_ID,X,Y,amphibians,SliceNumber,Dimensions
0,1,102,173.412225,-41.161038,1.0,3318,SliceNumber
1,2,102,173.412225,-41.161038,1.0,3393,SliceNumber
2,3,103,173.046567,-40.963196,1.0,3318,SliceNumber
3,4,104,172.595189,-41.794691,1.0,3318,SliceNumber
4,5,104,172.595189,-41.794691,1.0,3393,SliceNumber


In [23]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [26]:
frame = { 'amph_nspecies': amph, 'bird_nspecies': bir, 'mamm_nspecies': mamm, 'rept_nspecies': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['amph_nspecies', 'bird_nspecies', 'mamm_nspecies', 'rept_nspecies']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['amph_nspecies'] + df['bird_nspecies'] + df['mamm_nspecies'] + df['rept_nspecies']
df

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,0,49,3,0,52
1,2,2,68,2,6,78
2,3,2,24,2,2,30
3,4,2,62,2,4,70
4,5,2,48,0,3,53
...,...,...,...,...,...,...
205893,217453,2,95,25,2,124
205894,217455,2,76,25,2,105
205895,217456,0,40,2,0,42
205896,217457,6,59,40,3,108


In [27]:
df[df.nspecies==0]

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies


In [28]:
wdpa_updated = wdpa.merge(df, how='left', on = 'MOL_ID')
wdpa_updated

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,310492,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,0.0,49.0,3.0,0.0,52.0
1,1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,307797,Ferndale,Designated,NaN,Federal or national ministry or agency,2.0,68.0,2.0,6.0,78.0
2,2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,307745,Broughton Bay,Designated,NaN,Federal or national ministry or agency,2.0,24.0,2.0,2.0,30.0
3,3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,307867,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,2.0,62.0,2.0,4.0,70.0
4,4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,...,303963,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,2.0,48.0,0.0,3.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217481,217481,Sablayan Fish Sanctuary,217482,0.000000,555716517.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,555716517,Sablayan Fish Sanctuary,Established,2000.0,Not Reported,NaN,NaN,NaN,NaN,NaN
217482,217482,Gurion Marine Sanctuary,217483,0.000000,555716527.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,555716527,Gurion Marine Sanctuary,Established,2002.0,Not Reported,NaN,NaN,NaN,NaN,NaN
217483,217483,Bontod-Maravilla Fish Sanctuary and Marine Res...,217484,0.000000,555716306.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,555716306,Bontod-Maravilla Fish Sanctuary and Marine Res...,Established,2005.0,Not Reported,NaN,NaN,NaN,NaN,NaN
217484,217484,Bani Marine Protected Area,217485,0.000000,555716297.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,555716297,Bani Marine Protected Area,Established,2006.0,Not Reported,NaN,NaN,NaN,NaN,NaN


In [21]:
wdpa_updated.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs/wdpa_precal_with_contextual_nspecies_all.csv')

In [22]:
len(wdpa_updated)

217486

---